In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def load_deepfake_model(weights_path):
    xception_model = Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

    for layer in xception_model.layers[:-4]:
        layer.trainable = False

    output = xception_model.output
    output = GlobalAveragePooling2D()(output)
    output = Dense(256, activation='relu')(output)
    output = Dense(1, activation='sigmoid')(output)

    model = Model(inputs=xception_model.input, outputs=output)

    model.load_weights(weights_path)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

class DeepFakeDetectionModel:
    """Class to handle deepfake detection with a pretrained Xception model.
    """
    def __init__(self, path_to_weights):
        self.model = load_deepfake_model(path_to_weights)

    def predict(self, image_array):
        """Predict whether an image is a deepfake.
        Args:
            image_array (np.array): Preprocessed image array ready for prediction.

        Returns:
            float: The probability of the image being a deepfake.
        """
        return self.model.predict(image_array)

if __name__ == "__main__":
    model = DeepFakeDetectionModel("xception_weights.h5")
    prediction = model.predict(test_images)
    print("Prediction: ", prediction)


In [1]:
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import Xception
import base64
from io import BytesIO
from PIL import Image

app = Flask(__name__)

def load_deepfake_model(weights_path):
    xception_model = Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    for layer in xception_model.layers[:-4]:
        layer.trainable = False

    output = xception_model.output
    output = GlobalAveragePooling2D()(output)
    output = Dense(256, activation='relu')(output)
    output = Dense(1, activation='sigmoid')(output)
    model = Model(inputs=xception_model.input, outputs=output)
    model.load_weights(weights_path)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = load_deepfake_model('xception_weights.h5')

def decode_and_preprocess_image(base64_image):
    image_bytes = base64.b64decode(base64_image)
    img = Image.open(BytesIO(image_bytes))
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    base64_image = data['image']
    image_array = decode_and_preprocess_image(base64_image)
    prediction = model.predict(image_array)
    probability = prediction[0][0]
    return jsonify({'probability': float(probability)})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


83683744/83683744 [==============================] - 36s 0us/step
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Asus\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
